# Creating new feature Group

In [ ]:

import hopsworks
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('../../data/hopsworkKey.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1170584
Connected. Call `.close()` to terminate connection gracefully.


# Get data from hopswork

In [18]:
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
# query = air_quality_fg.select(["date", "pm25",])
query = air_quality_fg.select_all()
df_aq = query.read()

new_df = df_aq.copy()  
new_df.sort_values(by="date", inplace=True)
new_df.set_index("date", inplace=True)
new_df['pm25_rolling_mean'] = new_df['pm25'].rolling(window=3).mean() 
new_df.reset_index(inplace=True)

new_df['pm25'] = new_df['pm25_rolling_mean']  
new_df = new_df.drop(columns=['pm25_rolling_mean'])
new_df = new_df.rename(columns={ 'pm25':'pm25_rolling_mean'})
new_df.dropna(inplace=True)

new_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.99s) 


,date,pm25_rolling_mean,country,city,street,url
2,2018-06-30 00:00:00+00:00,78.666667,china,chengdu,chengdu-jinbo-road,https://api.waqi.info/feed/@10444
3,2018-07-01 00:00:00+00:00,74.000000,china,chengdu,chengdu-jinbo-road,https://api.waqi.info/feed/@10444
4,2018-07-02 00:00:00+00:00,50.666667,china,chengdu,chengdu-jinbo-road,https://api.waqi.info/feed/@10444
5,2018-07-03 00:00:00+00:00,44.666667,china,chengdu,chengdu-jinbo-road,https://api.waqi.info/feed/@10444
6,2018-07-04 00:00:00+00:00,42.000000,china,chengdu,chengdu-jinbo-road,https://api.waqi.info/feed/@10444
...,...,...,...,...,...,...
2290,2024-11-13 00:00:00+00:00,60.333333,china,chengdu,chengdu-jinbo-road,https://api.waqi.info/feed/@10444
2291,2024-11-14 00:00:00+00:00,71.666667,china,chengdu,chengdu-jinbo-road,https://api.waqi.info/feed/@10444
2292,2024-11-15 00:00:00+00:00,72.666667,china,chengdu,chengdu-jinbo-road,https://api.waqi.info/feed/@10444
2293,2024-11-16 00:00:00+00:00,63.000000,china,chengdu,chengdu-jinbo-road,https://api.waqi.info/feed/@10444


# Add expectations

In [30]:
import great_expectations as ge
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25_rolling_mean",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25_rolling_mean", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

In [31]:
air_pm25rolling_fg = fs.get_or_create_feature_group(
    name='air_pm25rolling',
    description='Air Quality air_pm25Rolling of past 3 days',
    version=1,
    primary_key=['city', 'street', 'date'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

In [32]:
air_pm25rolling_fg.insert(new_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1170584/fs/1161287/fg/1352029
2024-11-17 21:54:43,421 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1170584/fs/1161287/fg/1352029


Uploading Dataframe: 0.00% |          | Rows 0/2293 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_pm25rolling_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1170584/jobs/named/air_pm25rolling_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x228d3fb62e0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25_rolling_mean",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 675866
         }
       },
       "result": {
         "observed_value": 21.666666666666668,
         "element_count": 2293,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-17T08:54:43.000420Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successfu